In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra
import pandas as pd
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")


from sklearn.metrics import accuracy_score, confusion_matrix,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, LabelEncoder,PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Attribute Information:
1. date : Date in MM-DD-YYYY
2. day : Day of the Week
3. quarter : A portion of the month. A month was divided into four quarters
4. department : Associated department with the instance
5. teamno : Associated team number with the instance
6. noofworkers : Number of workers in each team 
7. noofstylechange : Number of changes in the style of a particular product
8. targetedproductivity : Targeted productivity set by the Authority for each team for each day.
9. smv : Standard Minute Value, it is the allocated time for a task 
10. wip : Work in progress. Includes the number of unfinished items for products 
11. overtime : Represents the amount of overtime by each team in minutes
12. incentive : Represents the amount of financial incentive (in BDT) that enables or motivates a particular course of action.
13. idletime : The amount of time when the production was interrupted due to several reasons 
14. idlemen : The number of workers who were idle due to production interruption
15. actual_productivity : The actual % of productivity that was delivered by the workers. It ranges from 0-1.

In [ ]:
df = pd.read_csv('../input/productivity-prediction-of-garment-employees/garments_worker_productivity.csv')

In [ ]:
df.sample(10)

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
df.describe(include=['O'])

In [ ]:
df.describe().T

# Data wrangling

In [ ]:
df['date'] = pd.to_datetime(df['date'])

In [ ]:
df['department'] = df['department'].apply(lambda x: 'finishing' if x == ('finishing ' or 'finishing' ) else 'sewing' )

In [ ]:
df['wip'].fillna(int(df['wip'].mean()), inplace=True)

In [ ]:
df.isnull().sum()

**No missing values now**

In [ ]:
sns.pairplot(df)

In [ ]:
sns.displot(df['actual_productivity'])

In [ ]:
df.actual_productivity.hist()

In [ ]:
plt.figure(figsize=(5,5))
sns.boxplot(data=df,y='actual_productivity')

In [ ]:
df.skew().sort_values(ascending=True)

In [ ]:
from scipy.stats.mstats import normaltest
normaltest(df.actual_productivity.values)

In [ ]:
log_price = np.log(df.actual_productivity)
log_price.hist()

In [ ]:
normaltest(log_price)

In [ ]:
from scipy.stats import boxcox
bc_price= boxcox(df.actual_productivity)
boxcox_price = bc_price[0]
lambd = bc_price[1]
plt.hist(boxcox_price)  

In [ ]:
normaltest(boxcox_price)

In [ ]:
plt.figure(figsize=(12,8))
sns.heatmap(df.corr(),annot=True)

In [ ]:
data=df.copy()

# Multiple linear Regression

In [ ]:
le = LabelEncoder()
data['department'] = le.fit_transform(data['department'])


X=data[['targeted_productivity', 'team','smv','idle_men', 'no_of_style_change']]
X1 = data[['team', 'targeted_productivity','smv', 'wip', 'over_time', 'incentive', 'idle_men','no_of_style_change', 'no_of_workers']]
y=data['actual_productivity']

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=42)


s = StandardScaler()
X_train_sm = s.fit_transform(X_train)

mlr = LinearRegression()
mlr.fit(X_train_sm, y_train)
X_test_sm = s.transform(X_test)
y_pred_sm = mlr.predict(X_test_sm)

In [ ]:
X1.head()

In [ ]:
X1['department'].value_counts()

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks

In [ ]:
le = LabelEncoder()
data['department'] = le.fit_transform(data['department'])

X1 = data[['team', 'targeted_productivity','smv', 'wip', 'over_time', 'incentive', 'idle_men','no_of_style_change', 'no_of_workers']]
y=data['actual_productivity']

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=42)


s = StandardScaler()
X_train_sm = s.fit_transform(X_train)

In [ ]:
input_shape = [X_train_sm.shape[1]]
model = keras.Sequential([
    layers.Dense(5, input_shape=input_shape, activation = 'relu'),
    layers.Dense(32, activation = 'relu'),
    layers.Dense(16, activation = 'relu'),
    layers.Dense(1, activation='sigmoid')
])
model.compile(
    optimizer = 'adam', 
    loss = "mean_absolute_error", 
    metrics=['accuracy'])
history = model.fit(X_train_sm, y_train,
          validation_data = (X_test, y_test),
          epochs=25)
# mean_squared_error  mean_absolute_error

In [ ]:
#Accuracy and MSE
print(f'R2 score is {r2_score(y_pred_sm,y_test)}')
print("Residual sum of squares (MSE): %.2f" % np.mean((y_pred_sm - y_test) ** 2))

In [ ]:
print(f'Co-efficients are {mlr.coef_}')
print(f'Intercept is {mlr.intercept_}')